In [1]:
from IPython.core.display import HTML
with open('./style.css') as f:
    css = f.read()
HTML(css)

# Prerequisites

In [26]:
import chess                                       # Simulate the chess game
from IPython.display import display, clear_output  # Better visualization and display of the chess board
import random                                      # Random moves and random creation of endgame positions
import pickle
import os.path

ModuleNotFoundError: No module named 'chess'

In [2]:
DIRECTORIES = {
    'S_SETS': 's_sets',
    'TABLEBASES': 'tables'
}

IndentationError: expected an indented block (2732712889.py, line 5)

## Prerequisites
First we need a further definition which we will use in our algorithm. The function
$next\_states: BOARD \Rightarrow 2^{BOARD} \\ $ will give us all possible boards which can be reached in only one half-move on a given board.


## Retrograde Analysis in Theory
Then we can begin to specify Retrograde Analysis. We will look at only a given endgame $E \in ENDGAME$ for which we hope to generate a tablebase. The goal is to determine a value $n$ for each chess board $b \in E$, which holds the number of half-moves needed to checkmate the opponent. That way we can always choose the move leading to a lower value of $n$, bringing the game closer to a checkmate. To assign the value, we will start at $n=0$, which is checkmate, and then move backwards, exploring all possible moves one by one, while incrementing $n$. If successfull, a great number of possible boards can directly lead to a checkmate. 

To do this we will sort the representations of all possible boards $b \in E$ into a series of sets $(S_n)_{k} \bigcup S$ where k is the maximum value of half-moves needed to force a checkmate for the specific endgame. We will develop the series of sets as follows:

1. We will calculate all possible boards for an endgame $E$ and place them into the set $S$:
    $S := \{ b | b \in E\}$
2. Next, we will search for all boards in which the player to move is checkmate. These will be inserted into $S_0$ and removed from $S$.
    $S_0 := \{ b | b \in S \land is\_checkmate(b)\} \\ $ 
    $ S = S \diagdown S_0 $
    $ \\ $
    These boards can be interpreted as target states, in which the losing player will be forced into.
3. Then we can create all sets $S_n$ where $n$ is odd. These will hold all boards in which the player to move wins after at most $n$ half-moves (if played optimally).
    $ S_{2k+1} := \{ b_1 | b_1 \in S \land \exists b_2 \in S_{2k}:(b_2 \in next\_states(b_1)) \} \text{   with } k \in \mathbb{N}_0 \\ $
    $ S = S \diagdown S_{2k+1} $
    $ \\ $
    The set $S_1$ contains all boards in which the player to move has at least one move which leads into $S_0$. Therefore this is the winners last move. Trivially, all sets $S_n$ where $n$ is odd, will contain boards where its the winners turn to move. Therefore we only need to find a single move leading into $S_{n-1}$ which the player can choose.
4. The sets $S_n$ where $n$ is even hold all boards in which the player to move will lose after at most $n$ half-moves (if his opponent plays optimally).
    $ S_{2k} := \{ b_1 | b_1 \in S \land \forall b_2 \in S_{2j+1}:(b_2 \in next\_states(b_1)\} \text{   with } k,j \in \mathbb{N} \text{ and } j < k \\ $
    $ S = S \diagdown S_{2k}$
    $ \\$ 
    The sets $S_n$ with $n$ even, will hold boards in which the loser is to move. In order to force a checkmate upon him, all his available moves must lead into a set $S_k$ where $k < n$ and $k$ odd.
5. The proposed algorithm will continue to develop the next set according to the specifications until a set is found to be empty. 

### Interpreting the Results
Once the algorithm has terminated we have created a series of sets $(S_n)_n \bigcup S$. We can then redefine the depth-to-mate (DTM) according to our calculations for the Endgame. 
1. $ \forall r \in S_k:(|DTM(r)| = k)$. A victory can be forced in at most $k$ half-moves.
2. $ \forall r \in S:(DTM(r) = 0)$. In this case none of the players can force a victory by playing optimally. If the board is already checkmate or stalemate, we also assign a DTM value of zero.

## Board representation
In order to build a large tablebase capable of storing hundreds of thousands of possible board positions, it is important to choose an effective representation of a board to store. At first, one possibility might be to store the board using the FEN-string notation, however, strings use a large amount of memory. Encoding a single board in a smaller datatype, makes it possible for us to do computations with larger datasets in memory. Therefore we shall store our information of a Board within a single integer, which in Python amounts to 28 bytes of memory. 

**<mark>Definition (Representation)</mark>**

The function 

$$ \text{board_to_int}: Board \times \text{piece_string} \Rightarrow Integer $$ converts a board into its <span style="color:blue">Representation</span>. 

It is bijective with its inverse function $ int\_to\_board: Integer \times piece\_string \Rightarrow Board $.

### Implementations

When implementing the functions, the basic idea is to assign a byte within a python bytearray to each piece on the board. The value held by the byte is the position of the piece, where each field on the board is given an value between 0 and 63. The <b><code>piece_map</code></b> method within python-chess returns a dictionary containing each piece and its field. We shall use this dictionary to iterate over the chess pieces and assign the bytes. If a piece has been captured the corresponding byte will be set to <b>0xff</b>. The last byte will hold the turn information, which is stored as well as either one or zero. Finally, the resulting bytearray is encoded as a single integer and returned. 

In [12]:
def board_to_int(board, piece_str):
    piece_map = board.piece_map()
    representation = bytearray(len(piece_str) + 1)
    for i, piece_symbol in enumerate(piece_str):
        position_list = [pos for pos, piece in piece_map.items() if piece == chess.Piece.from_symbol(piece_symbol)]
        if not position_list: 
            representation[i] = 0xFF
        else:
            piece_map.pop(position_list[0])
            representation[i] = position_list[0]
    representation[-1] = board.turn
    return int.from_bytes(representation, 'little')

<h3>Decoding</h3>
<div style="text-align: justify">
Similarly, to decode an integer, the bytearray is transformed back into a piece mapping. The resulting dictionary is used to initialize a new chess board, which is returned. 
</div>

In [13]:
def int_to_board(representation, piece_str):
    board = chess.Board(None)
    byte_representation = representation.to_bytes(len(piece_str)+1, "little")
    mapping = dict(zip(byte_representation[:-1], [chess.Piece.from_symbol(piece_symbol) for piece_symbol in piece_str]))
    for pos in mapping.keys():
        if pos == 0xFF: mapping.pop(pos)
    board.set_piece_map(mapping)
    board.turn = byte_representation[-1]
    return board

To demonstrate this, we will:
1. create a board from a given FEN string 
2. encode it as an integer
3. decode it back into a board and then 
4. read out the FEN string.

In [14]:
fen = '3K4/8/4k3/8/8/8/8/2r5 b - - 0 1'
board = chess.Board(fen)
representation = board_to_int(board, 'Kkr')
new_board = int_to_board(representation, 'Kkr')
print(f"FEN to INT: '{fen}' -> {representation}")
print(f"INT to FEN: {representation} -> '{new_board.fen()}'")

NameError: name 'chess' is not defined

# The Algorithm

## 1. Calculating All Possible Permutations
Initially we have to calculate a set $S$ containing all boards for a given endgame $E$. We use the notation $S^P$ for the set $S$ of the endgame with piece_string $P$ (Ex: $S^{Kkr}$). For efficiency we want to be able to save $S$ for a given endgame to disk and load it back into memory if it is needed. So we will first define two auxiliary functions:
1. $save\_s: PIECE\_STRING \times 2^E \Rightarrow None$ will save the set $S$ as a binary file using the pickle module. 
2. $load\_s: PIECE\_STRING \Rightarrow 2^E \bigcup \varnothing$ will attempt to load the set $S$ from a binary file. If it fails it will return an empty set.

In [15]:
def save_s(piece_str, s):
    path = f'./{DIRECTORIES["S_SETS"]}/{piece_str}'
    with open(path, 'wb') as f:
        pickle.dump(s, f)

In [16]:
def load_s(piece_str):
    path = f'./{DIRECTORIES["S_SETS"]}/{piece_str}'
    if os.path.exists(path):
        with open(path, 'rb') as f:
            print(f"Loading s from {path}")
            return pickle.load(f)
    return set()

Furthermore, an endgame can be either 3-man or 4-man. Since a piece may also be taken during the game, the set $S$ of a 4-man endgame does also contain all boards of a set $S$ of a 3-man endgame if the three pieces are part of the 4-man piece_string. 
For example $S^{KBNk}$ also contains the boards of both $S^{KBk}$ and $S^{KNk}$. Therefore we shall define a function 
$ split\_into\_substrings: piece\_string \Rightarrow 2^{piece\_string} \\ $
which tells us which other sets we have to calculate as well. 

In [17]:
def split_into_substrs(piece_str):
    substrings = {piece_str}
    if len(piece_str) == 4:
        non_king_pieces = piece_str.replace('K', '').replace('k', '')
        for p in non_king_pieces:
            substrings.add(piece_str.replace(p, '', 1))
    return substrings

To demonstrate this, we want to list the substrings of the following endgame piece_strings:

In [18]:
endgames = ['KRk', 'KQk', 'KBBk', 'KNNk', 'KBNk', 'KQkr', 'Kkr', 'Kkq', 'Kkbb', 'Kknn', 'Kkbn', 'KRkq']
for e in endgames:
    print("{:^4} => {}".format(e, str(split_into_substrs(e))))

KRk  => {'KRk'}
KQk  => {'KQk'}
KBBk => {'KBBk', 'KBk'}
KNNk => {'KNNk', 'KNk'}
KBNk => {'KNk', 'KBk', 'KBNk'}
KQkr => {'Kkr', 'KQk', 'KQkr'}
Kkr  => {'Kkr'}
Kkq  => {'Kkq'}
Kkbb => {'Kkbb', 'Kkb'}
Kknn => {'Kkn', 'Kknn'}
Kkbn => {'Kkn', 'Kkb', 'Kkbn'}
KRkq => {'KRkq', 'KRk', 'Kkq'}


We will now go on to generate all boards for a given substring of the piece_string. In order to count through all possible positions for the pieces on the board, we will implement a simple helper function $shift\_positions$ which will receive a list through which it counts by incrementing one.

In [19]:
def shift_positions(positions):
    for i, pos in enumerate(positions[::-1]):
        if pos == 63: 
            positions[-(i+1)] = 0
            continue
        else:
            positions[-(i+1)] += 1
            break
    return positions

We then define $ generate\_sub\_s: PIECE\_STRING \Rightarrow 2^E \\ $
First we have to check if we already calculated the relevant set, and if so we can load it from the disk. If not, we proceed with the calculations. The algorithm iterates over each turn, (black and white) and further iterates over all permutations of field positions. Each permutation is mapped onto the piece_string and loaded onto a board. Next, we must check if it is a valid board using the $is\_valid$ method of the chess library. If so, we can encode the board as an int and add it to the set $S$. 

<b>Remark:</b> Within this function we included print statements showing the current progress as the generation process may take several minutes. This has been done for several of the following functions as well.

In [20]:
def generate_sub_s(piece_str):
    
    print(f"[+] Generating {piece_str}")
    
    sub_s = load_s_if_exists(piece_str)
    
    if sub_s == set():
        board = chess.Board(None)
        for turn in [chess.WHITE, chess.BLACK]:
            board.turn = turn
            positions = [0] * len(piece_str)
            while positions != [63] * len(piece_str):
                if positions[-1] == 63 and positions[-2] == 63:
                    print("\r", end="")
                    print(f"{positions} -> {int(100/126*(positions[0] if turn == chess.WHITE else positions[0]+63))}%", end="")

                mapping = dict(zip(positions, [chess.Piece.from_symbol(piece) for piece in piece_str]))
                board.set_piece_map(mapping)
                if board.is_valid() and len(set(positions)) == len(piece_str):
                    sub_s.add(board_to_int(board, piece_str))
                positions = shift_positions(positions)
                board.clear_board()
    
        if len(piece_str) == 3:
            save_s(piece_str, sub_s)
                
    return sub_s

However, if the piece_string has a length of 4 we must also calculate the sets $S$ for the substrings of the piece_string and add them. $ generate\_s: PIECE\_STRING \Rightarrow 2^E \\ $ Putting it all together, this function finds the substrings of the Piece_String, calculates $S$ for each of them, saves the union of those sets to disk $S$ and returns it.

In [8]:
def generate_s(piece_str):
    s = load_s(piece_str)
    if s == set():
        substrs = split_into_substrs(piece_str)
        print(f"{piece_str} needs the following substrs: {substrs}")
        for substr in substrs:
            s.union(generate_sub_s(substr))
        if len(piece_str) == 4:
            save_s(piece_str, s)
    return s

## 2. Calculating All Mate Positions
The next step is to calculate all boards within $S$, where the player to move is checkmate. These positions will be stored in the set $S_0$ and subtracted from $S$. We implement the function:
$generate\_mate\_positions: 2^E \times Piece\_String \Rightarrow 2^S \times 2^S \\ $
The function takes the $S$ set and the $Piece\_String$ and returns the new $S$ and the generated $S_0$. Its implementation relies on the $is\_checkmate$ method of the chess library. 


In [28]:
def generate_mate_positions(s, piece_str):
    s_0 = set()
    for i, representation in enumerate(s):
        if i%1000 == 0: 
            print("\r", end="")
            print(f"{i}/{len(s)} -> {int((i/len(s))*100)}%", end="")
        if int_to_board(representation, piece_str).is_checkmate():
            s_0.add(representation)  
    s = set(s) - s_0
    return s, s_0

## 3. Generating $S_n$ where $n$ is odd
In order to generate the sets with an odd $n$, we have to implement the specification
$ S_{2k+1} := \{ b_1 | b_1 \in S \land \exists b_2 \in S_{2k}:(b_2 \in next\_states(b_1)) \} \text{   with } k \in \mathbb{N}_0 \\ $
We define the function $generate\_s\_odd : 2^S \times 2^S \times Piece\_String \Rightarrow 2^S \times 2^S \\ $
which takes 
1. The current $S$
2. The previously calculated set $S_{n-1}$
3. A $Piece\_String$. 

For purpose of efficiency, the implementation does not rely on a $next\_states$ function. Instead while iterating through $S$, the next legal move of each board is queried and applied using the chess library. Once a resulting board is found in $S_{n-1}$ as well, it is added to $S_n$. Finally, we can subtract $S_n$ from $S$ and return both.

In [22]:
def generate_s_odd(s, s_prev, piece_str):
    s_next = set()
    
    for i, representation in enumerate(s):

        if i%1000 == 0: 
            print("\r", end="")
            print(f"{i}/{len(s)} -> {int((i/len(s))*100)}%", end="")
        
        board = int_to_board(representation, piece_str)
        for move in board.legal_moves:
            board.push(move)

            if board_to_int(board, piece_str) in s_prev:
                board.pop()
                s_next.add(board_to_int(board, piece_str))
                break

            board.pop()

    s = s - s_next
    return s, s_next

## 4. Generating $S_n$ where $n$ is even
The next function follows a similar specification. This time however it is the losers turn. Therefore we must ensure that all possible moves lead into a state closer to checkmate, before adding the board to our next $S_n$.
$ S_{2k} := \{ b_1 | b_1 \in S \land \forall b_2 \in S_{2j+1}:(b_2 \in next\_states(b_1)\} \text{   with } k,j \in \mathbb{N} \text{ and } j < k \\ $
$ S = S \diagdown S_{2k}$

We define a function 
$generate\_s\_even : 2^S \times (2^S)^{k-1} \times Piece\_String \Rightarrow 2^S \times 2^S \\ $ 
which takes 
1. The current $S$ 
2. A list of previously calculated sets $S_{n}$ with $n$ odd and $n < 2k$
3. A $Piece\_String$.

Similarly to above, we shall iterate through $S$, calculating the next boards for each board, by applying all legal moves. Then, only if all resulting boards lead into a board within $S_{2j+1}$ with $j < k$ the original board will be added to $S_{2k}$ which will become the next $S_n$.

In addition, we have to check that we do not end in a stalemate, where the player has no legal moves left. 

Lastly, we will once again subtract $S_n$ from $S$.

In [23]:
def generate_s_even(s, l_odd, piece_str):
    s_next = set()

    for i, representation in enumerate(s):

        if i%1000 == 0: 
            print("\r", end="")
            print(f"{i}/{len(s)} -> {int((i/len(s))*100)}%", end="")

        board = int_to_board(representation, piece_str)
        legal_moves = board.legal_moves
        
        
        fail = not legal_moves #no legal moves for stalemate position
        
        for move in legal_moves:
            board.push(move)

            if not any(board_to_int(board, piece_str) in s_odd for s_odd in l_odd):
                fail = True
                break

            board.pop()
        
        if not fail:
            s_next.add(board_to_int(board, piece_str))

    s = s - s_next
    return s, s_next

## 5. Constructing the Main Algorithm

Before we can implement the main routine, we shall define a function to call at the very end once we finished calculating all sets. Its job is to take the final list of sets and save it to disk as a binary. The resulting file is effectively a tablebase where we can determine the DTM for each board, by looking in which set $S_n$ it is: 
$ \forall b \in S: ( b \in S_n \Rightarrow DTM(b) = n) \\ $ 

In [25]:
def save_sets_as_binary(list_of_sets, piece_str):
    with open(f"./{DIRECTORIES['TABLEBASES']}/{piece_str}", "wb") as f:
        pickle.dump(list_of_sets, f)

The main function will now progressively develop our series of sets for a given $Piece\_String$. It will start by generating $S$. Then it enters a while loop, increasing the variable $n = 0$ after every iteration and generating the next $S_n$. Inside the loop we have 4 cases, in which case the $S$ and $S_n$ are overwritten by the next set.:
1. If $n = 0$ we shall generate $S_0$ using the $generate\_mate\_positions$ function.  
2. If $n$ is odd, we will generate $S_n$ using $generate\_s\_odd$ and supply the set $S_{n-1}$ as an argument.
3. If $n$ is even, we will generate $S_n$ using $generate\_s\_even$ and supply every previously calculated set $S_n$ with $n$ odd as an argument.
4. If we discover that after one of the previous cases, $S_n$ is empty, we have reached the last set and can break out of the loop. 

After every successful iteration, the new set $S_n$ is added to the $list\_of\_sets$. 

Once the loop has been terminated, we can save the resulting $list\_of\_sets$ to disk. 


Currently we are storing all sets in memory before the function has terminated, which can take up close to 5GB of RAM for a 4-man tablebase. The execution speed varies from machine to machine, but a 3-man tablebase can be created in under 24h using the free Deepnote cloud computing service. 


In [37]:
def main(piece_str):
    
    print(f"Doing all positions")
    s = generate_s(piece_str)
    print(f"\nLength of s = {len(s)}")
    print(f"--------------------------------------------------------------------------------------------")

    list_of_sets = []

    n = 0
    
    while True: 

        print(f"Doing s_{n}")

        if n == 0: 
            s, s_n = generate_mate_positions(s, piece_str)
        elif n%2 == 1: 
            s, s_n = generate_s_odd(s, list_of_sets[-1], piece_str)
        else: 
            s, s_n = generate_s_even(s, list_of_sets[1::2], piece_str)
        
        if not s_n: break
        
        list_of_sets.append(s_n)

        print(f"\nLength of s_{n} = {len(s_n)}")
        print(f"New length of s = {len(s)}")
        print(f"--------------------------------------------------------------------------------------------")

        n += 1 

    save_sets_as_binary(list_of_sets, piece_str)

In [38]:
#main('Kknb')